In [1]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [11]:
data = pd.read_csv('../dataset/cleaned-dataset.csv')
print("Dataset loaded:", data.shape)


Dataset loaded: (11162, 17)


In [15]:
# Step 3: Separate features and target
# The correct target column is 'deposit', not 'y_yes'
X = data.drop('deposit', axis=1)
y = data['deposit']

print("Features (X) shape:", X.shape)
print("Target (y) shape:", y.shape)

Features (X) shape: (11162, 16)
Target (y) shape: (11162,)


In [16]:
# Step 4: Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [17]:
# Step 5: Apply KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=42)
data['Cluster'] = kmeans.fit_predict(X_scaled)

In [18]:
# Save KMeans model
with open('kmeans_model.pkl', 'wb') as f:
    pickle.dump(kmeans, f)
print("KMeans model saved.")

KMeans model saved.


In [20]:
# Step 6: Train Logistic Regression per cluster
logistic_models = {}
accuracies = []

for cluster_id in sorted(data['Cluster'].unique()):
    cluster_data = data[data['Cluster'] == cluster_id]
    
    # Correction 1: Change 'y_yes' to 'deposit'
    X_cluster = cluster_data.drop(['deposit', 'Cluster'], axis=1)
    
    # Correction 2: Change 'y_yes' to 'deposit'
    y_cluster = cluster_data['deposit']
    
    X_train, X_test, y_train, y_test = train_test_split(X_cluster, y_cluster, test_size=0.2, random_state=42)
    
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'Cluster {cluster_id} Accuracy: {acc:.2f}')
    
    logistic_models[cluster_id] = model
    accuracies.append(acc)

# ... rest of the code

c:\Users\RIDA\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Cluster 0 Accuracy: 0.76


c:\Users\RIDA\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Cluster 1 Accuracy: 0.79
Cluster 2 Accuracy: 0.88


c:\Users\RIDA\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
# Save all logistic models
with open('logistic_models.pkl', 'wb') as f:
    pickle.dump(logistic_models, f)
print("Logistic models saved.")


Logistic models saved.


In [22]:
# Step 7: Overall Hybrid Accuracy
print("Average Hybrid Model Accuracy:", np.mean(accuracies))


Average Hybrid Model Accuracy: 0.8068299679031407
